In [1]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [2]:
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format

# Get the NIHTB Mater data

In [3]:
import pyarrow.parquet as pq

df = pq.read_table(source='R:/MSS/Research/NIHTB v3.0 Norming/SG Data/Cleaned_Data/2022_04_08/2022-04-08_B_NIHTB_Master.parquet.gzip').to_pandas()
df.shape

(8173451, 17)

In [5]:
#Filter Retest_order == FALSE or first; there should be NO duplicates. 
df1 = df[df.Retest_order != 'second']
df1.shape

(7759595, 17)

In [6]:
df1.TestName.unique()

array(['PV_norm', 'Flank_norm', 'LSWM_norm', 'DCCS_norm', 'PC_norm',
       'PSM_norm', 'OR_norm', 'AudLrn_norm', 'VR_norm', 'OSD_norm',
       'StandBal_norm', 'FNAME_norm', 'SpdMtch_norm', 'DCCS_SAdmin',
       'PicSeq_SAdmin'], dtype=object)

# List Sort Working Memory

In [351]:
def select_Test(df, tname):
    long = df[df.TestName==tname]
    
    #Reshape the data from long to wide
    wide=long.pivot_table(index=['FinalPIN', 'DeviceID', 'AssessmentName', 'ResponseID', 'ItemID',
       'InstrumentTitle', 'TestName', 'InstrumentID','age_months_at_test'], columns='Key', values='Value', aggfunc='first').reset_index()
    #sort the dataframe by DateCreated
    #Convert to datetime
    wide['DateCreated']= pd.to_datetime(wide['DateCreated'], format='%Y-%m-%d %H:%M:%S')
    wide =wide.sort_values('DateCreated')
    
    return wide
    

In [354]:
ldf_wide = select_Test(df1, 'LSWM_norm')

In [355]:
ldf_wide.head()

Key,FinalPIN,DeviceID,AssessmentName,ResponseID,ItemID,InstrumentTitle,TestName,InstrumentID,age_months_at_test,DateCreated,Response,Score
19674,MSL19002,5CECD629-9F82-40E5-A7C0-05014145BC4C,Assessment 1,BC858299-FCBA-4051-B263-3614409ED472,LSWM_Test_Name,NIH Toolbox LSWM Norming Version Form A,LSWM_norm,938EFF0C-651F-4063-BC3F-A775E32FE443,238.20,2021-06-02 08:42:59,1,NaN
19656,MSL19002,5CECD629-9F82-40E5-A7C0-05014145BC4C,Assessment 1,29B1A183-748B-45BF-B39B-0A157818DCD0,LSWM_Intro_1,NIH Toolbox LSWM Norming Version Form A,LSWM_norm,938EFF0C-651F-4063-BC3F-A775E32FE443,238.20,2021-06-02 08:43:23,1,NaN
19670,MSL19002,5CECD629-9F82-40E5-A7C0-05014145BC4C,Assessment 1,8FA175DA-470F-42E6-884E-799EC3057F1F,LSWM_1List_Pract1_Trial1,NIH Toolbox LSWM Norming Version Form A,LSWM_norm,938EFF0C-651F-4063-BC3F-A775E32FE443,238.20,2021-06-02 08:43:36,1,1
19678,MSL19002,5CECD629-9F82-40E5-A7C0-05014145BC4C,Assessment 1,EE7D76EA-316D-46B6-B558-175CFB966F21,LSWM_ReadyIntro,NIH Toolbox LSWM Norming Version Form A,LSWM_norm,938EFF0C-651F-4063-BC3F-A775E32FE443,238.20,2021-06-02 08:43:44,1,NaN
19661,MSL19002,5CECD629-9F82-40E5-A7C0-05014145BC4C,Assessment 1,4CE95452-2729-4982-8690-9F5C4514DF18,LSWM_1List_Pract2_Trial1,NIH Toolbox LSWM Norming Version Form A,LSWM_norm,938EFF0C-651F-4063-BC3F-A775E32FE443,238.20,2021-06-02 08:43:56,1,1


# a.Time_List1 + e. N_List1

In [269]:
#Sanity check
#NOTE: There should be 3537 unique records for LSWM 
ldf_wide.FinalPIN.nunique()

3537

In [303]:
def get_Live_Item_Duration(df, List = 1):
    if List == 1:
        Interest = 'LSWM_1List_Live_Item'
    else:
        Interest ='LSWM_2List_Live_Item'
    drop = '_Live_Item_Trans'
    
    
    #After filter by List1 live item there is only 3498 unique PIN
    data= df[df.ItemID.str.startswith(Interest)]
    #drop the rows with Item_Trans
    data = data[~data.ItemID.str.contains(drop)]
    nlist =data.groupby('FinalPIN')['ItemID'].count().reset_index()
    nlist.columns =['FinalPIN', 'N_List'+str(List)]
    
    first=data.groupby('FinalPIN')['ItemID','DateCreated'].first()
    last =data.groupby('FinalPIN')['ItemID','DateCreated'].last()
    
    t1 = pd.merge(first, last, on='FinalPIN', how='outer')
    t1.columns =['ItemID_first'+str(List), 'DateCreated_first'+str(List), 'ItemID_last'+str(List), 'DateCreated_last'+str(List)]
    #Calculate the duration
    t1['Time_list'+str(List)] =pd.to_datetime(t1['DateCreated_last'+str(List)], format='%Y-%m-%d %H:%M:%S')-pd.to_datetime(t1['DateCreated_first'+str(List)], format='%Y-%m-%d %H:%M:%S')
    rv = pd.merge(t1, nlist, on='FinalPIN',how='outer')
    display(rv.head())  
    
    return rv

In [305]:
t1= get_Live_Item_Duration(ldf_wide, 1)

,FinalPIN,ItemID_first1,DateCreated_first1,ItemID_last1,DateCreated_last1,Time_list1,N_List1
0,MAP05001,LSWM_1List_Live_Item_A,2021-06-13 11:59:21,LSWM_1List_Live_Item_H,2021-06-13 12:01:36,0 days 00:02:15,6
1,MAP05003,LSWM_1List_Live_Item_A,2021-06-10 19:48:08,LSWM_1List_Live_Item_H,2021-06-10 19:49:14,0 days 00:01:06,6
2,MAP05004,LSWM_1List_Live_Item_B,2021-06-06 13:36:32,LSWM_1List_Live_Item_G,2021-06-06 13:38:03,0 days 00:01:31,6
3,MAP05005,LSWM_1List_Live_Item_A,2021-06-20 13:33:16,LSWM_1List_Live_Item_L,2021-06-20 13:36:11,0 days 00:02:55,10
4,MAP05006,LSWM_1List_Live_Item_B,2021-06-10 20:40:01,LSWM_1List_Live_Item_G,2021-06-10 20:41:11,0 days 00:01:10,6


In [306]:
t1.shape

(3497, 7)

# b.Time_List2 + e. N_List2

In [307]:
t2 = get_Live_Item_Duration(ldf_wide, 2)

,FinalPIN,ItemID_first2,DateCreated_first2,ItemID_last2,DateCreated_last2,Time_list2,N_List2
0,MAP05001,LSWM_2List_Live_Item_O,2021-06-13 12:04:31,LSWM_2List_Live_Item_R,2021-06-13 12:05:26,0 days 00:00:55,4
1,MAP05003,LSWM_2List_Live_Item_O,2021-06-10 19:51:07,LSWM_2List_Live_Item_R,2021-06-10 19:51:44,0 days 00:00:37,4
2,MAP05004,LSWM_2List_Live_Item_P,2021-06-06 13:40:38,LSWM_2List_Live_Item_Q,2021-06-06 13:41:25,0 days 00:00:47,4
3,MAP05005,LSWM_2List_Live_Item_O,2021-06-20 13:37:28,LSWM_2List_Live_Item_T,2021-06-20 13:39:01,0 days 00:01:33,6
4,MAP05006,LSWM_2List_Live_Item_P,2021-06-10 20:42:26,LSWM_2List_Live_Item_Q,2021-06-10 20:42:59,0 days 00:00:33,4


In [308]:
t2.shape

(3456, 7)

In [309]:
comb =pd.merge(t1,t2, on ='FinalPIN', how='left')
comb.tail()

,FinalPIN,ItemID_first1,DateCreated_first1,ItemID_last1,DateCreated_last1,Time_list1,N_List1,ItemID_first2,DateCreated_first2,ItemID_last2,DateCreated_last2,Time_list2,N_List2
3492,WPX85109,LSWM_1List_Live_Item_B,2021-08-21 12:46:06,LSWM_1List_Live_Item_I,2021-08-21 12:47:58,0 days 00:01:52,8,LSWM_2List_Live_Item_P,2021-08-21 12:49:15,LSWM_2List_Live_Item_W,2021-08-21 12:51:59,0 days 00:02:44,10.00
3493,XMAP13003,LSWM_1List_Live_Item_A,2021-06-07 16:38:36,LSWM_1List_Live_Item_L,2021-06-07 16:40:58,0 days 00:02:22,10,LSWM_2List_Live_Item_O,2021-06-07 16:42:36,LSWM_2List_Live_Item_V,2021-06-07 16:44:15,0 days 00:01:39,8.00
3494,XMSL14003,LSWM_1List_Live_Item_B,2021-06-09 08:44:14,LSWM_1List_Live_Item_I,2021-06-09 08:46:11,0 days 00:01:57,8,LSWM_2List_Live_Item_P,2021-06-09 08:47:24,LSWM_2List_Live_Item_W,2021-06-09 08:50:17,0 days 00:02:53,10.00
3495,XNBS10002,LSWM_1List_Live_Item_A,2021-06-10 14:13:12,LSWM_1List_Live_Item_N,2021-06-10 14:18:55,0 days 00:05:43,12,LSWM_2List_Live_Item_O,2021-06-10 14:20:15,LSWM_2List_Live_Item_Z,2021-06-10 14:25:58,0 days 00:05:43,12.00
3496,XNBS30005,LSWM_1List_Live_Item_A,2021-06-15 17:38:03,LSWM_1List_Live_Item_J,2021-06-15 17:40:12,0 days 00:02:09,8,LSWM_2List_Live_Item_O,2021-06-15 17:41:22,LSWM_2List_Live_Item_X,2021-06-15 17:44:20,0 days 00:02:58,10.00


# c. TimeTotal

In [310]:
start = ldf_wide[ldf_wide.ItemID.str.startswith('LSWM_Test_Name')][['FinalPIN','DateCreated','InstrumentTitle','age_months_at_test']]

In [311]:
start.columns=['FinalPIN', 'StartTime','InstrumentTitle','age_months_at_test']

In [312]:
start.shape

(3537, 4)

In [313]:
comb = pd.merge(start, comb,on='FinalPIN', how='right')

In [314]:
#TimeTotal = DateCreated of LSWM_Test_Name to the last LSWM_2_List_Live_Item_.  
comb['TimeTotal'] =comb['DateCreated_last2'] - comb['StartTime']

# d.Time_AllPractandTrans 

In [315]:
#Time_Total minus (Time_List1 + Time_List2) 
comb['Time_AllPractandTrans'] = comb['TimeTotal'] -comb['Time_list1'] - comb['Time_list2']

In [316]:
comb.head()

,FinalPIN,StartTime,InstrumentTitle,age_months_at_test,ItemID_first1,DateCreated_first1,ItemID_last1,DateCreated_last1,Time_list1,N_List1,ItemID_first2,DateCreated_first2,ItemID_last2,DateCreated_last2,Time_list2,N_List2,TimeTotal,Time_AllPractandTrans
0,MAP05001,2021-06-13 11:56:31,NIH Toolbox LSWM Norming Version Form A,71.03,LSWM_1List_Live_Item_A,2021-06-13 11:59:21,LSWM_1List_Live_Item_H,2021-06-13 12:01:36,0 days 00:02:15,6,LSWM_2List_Live_Item_O,2021-06-13 12:04:31,LSWM_2List_Live_Item_R,2021-06-13 12:05:26,0 days 00:00:55,4.00,0 days 00:08:55,0 days 00:05:45
1,MAP05003,2021-06-10 19:46:33,NIH Toolbox LSWM Norming Version Form A,72.38,LSWM_1List_Live_Item_A,2021-06-10 19:48:08,LSWM_1List_Live_Item_H,2021-06-10 19:49:14,0 days 00:01:06,6,LSWM_2List_Live_Item_O,2021-06-10 19:51:07,LSWM_2List_Live_Item_R,2021-06-10 19:51:44,0 days 00:00:37,4.00,0 days 00:05:11,0 days 00:03:28
2,MAP05004,2021-06-06 13:34:45,NIH Toolbox LSWM Norming Version Form B,66.01,LSWM_1List_Live_Item_B,2021-06-06 13:36:32,LSWM_1List_Live_Item_G,2021-06-06 13:38:03,0 days 00:01:31,6,LSWM_2List_Live_Item_P,2021-06-06 13:40:38,LSWM_2List_Live_Item_Q,2021-06-06 13:41:25,0 days 00:00:47,4.00,0 days 00:06:40,0 days 00:04:22
3,MAP05005,2021-06-20 13:32:01,NIH Toolbox LSWM Norming Version Form A,66.33,LSWM_1List_Live_Item_A,2021-06-20 13:33:16,LSWM_1List_Live_Item_L,2021-06-20 13:36:11,0 days 00:02:55,10,LSWM_2List_Live_Item_O,2021-06-20 13:37:28,LSWM_2List_Live_Item_T,2021-06-20 13:39:01,0 days 00:01:33,6.00,0 days 00:07:00,0 days 00:02:32
4,MAP05006,2021-06-10 20:38:39,NIH Toolbox LSWM Norming Version Form B,67.81,LSWM_1List_Live_Item_B,2021-06-10 20:40:01,LSWM_1List_Live_Item_G,2021-06-10 20:41:11,0 days 00:01:10,6,LSWM_2List_Live_Item_P,2021-06-10 20:42:26,LSWM_2List_Live_Item_Q,2021-06-10 20:42:59,0 days 00:00:33,4.00,0 days 00:04:20,0 days 00:02:37


In [317]:
keep =['FinalPIN',  'age_months_at_test','InstrumentTitle',
        'Time_list1',  'Time_list2','Time_AllPractandTrans', 'TimeTotal', 'N_List1','N_List2']

In [318]:
#sanity check
comb[comb.FinalPIN =='MAP09008'][keep]

,FinalPIN,age_months_at_test,InstrumentTitle,Time_list1,Time_list2,Time_AllPractandTrans,TimeTotal,N_List1,N_List2
38,MAP09008,118.67,NIH Toolbox LSWM Norming Version Form B,0 days 00:02:18,0 days 00:01:37,0 days 00:02:21,0 days 00:06:16,10,8.00


In [319]:
comb.shape

(3497, 18)

In [320]:
comb[keep].head()

,FinalPIN,age_months_at_test,InstrumentTitle,Time_list1,Time_list2,Time_AllPractandTrans,TimeTotal,N_List1,N_List2
0,MAP05001,71.03,NIH Toolbox LSWM Norming Version Form A,0 days 00:02:15,0 days 00:00:55,0 days 00:05:45,0 days 00:08:55,6,4.00
1,MAP05003,72.38,NIH Toolbox LSWM Norming Version Form A,0 days 00:01:06,0 days 00:00:37,0 days 00:03:28,0 days 00:05:11,6,4.00
2,MAP05004,66.01,NIH Toolbox LSWM Norming Version Form B,0 days 00:01:31,0 days 00:00:47,0 days 00:04:22,0 days 00:06:40,6,4.00
3,MAP05005,66.33,NIH Toolbox LSWM Norming Version Form A,0 days 00:02:55,0 days 00:01:33,0 days 00:02:32,0 days 00:07:00,10,6.00
4,MAP05006,67.81,NIH Toolbox LSWM Norming Version Form B,0 days 00:01:10,0 days 00:00:33,0 days 00:02:37,0 days 00:04:20,6,4.00


In [321]:
comb[keep].to_csv('C:/Users/tsb7592/Downloads/LSWM_Timing.csv', index=False)

In [327]:
td =comb.TimeTotal.loc[0]

In [332]:
minutes = td.total_seconds()//60
secs = td.total_seconds()%60
print(minutes, secs)

8.0 55.0


# Speeded Matching
## Part 1

In [356]:
sdf_wide = select_Test(df1, 'SpdMtch_norm')

In [357]:
#Using the same parquet dataset from List Sorting, but after filtering, there are only 846 cases. 
sdf_wide.FinalPIN.nunique()

846

# 1) Time Live 

In [346]:
drop =['Spdm_Title', 'SpdM_INTRO', 'SpdMIns1', 'SpdMPract_Transition',
       'SpdMP1', 'SpdMP2', 'SpdMP3', 'SpdMP4', 'Spdm_Transition']

In [360]:
#a. Select all ItemIDs that have the format “SpdM[numeric only]”
sdf_wide= sdf_wide[~sdf_wide.ItemID.isin(drop)]

In [368]:
sdf_wide['ResponseTime'] = pd.to_numeric(sdf_wide['ResponseTime'])
sdf_wide['Score'] = pd.to_numeric(sdf_wide['Score'])

In [372]:
sdf_wide.shape

(34649, 13)

In [424]:
#b. Add up all the ResponseTimes within these ItemIDs
s1 =sdf_wide.groupby(['FinalPIN','age_months_at_test'])['ResponseTime'].sum().reset_index()
s1

,FinalPIN,age_months_at_test,ResponseTime
0,MAP03001,43.07,105.94
1,MAP03002,41.82,91.20
2,MAP03003,43.73,92.61
3,MAP03004,37.49,102.17
4,MAP03005,39.75,93.32
...,...,...,...
837,WPX08020,99.91,92.57
838,WPX08021,105.40,91.21
839,WPX08022,107.40,91.91
840,WPX10003,128.40,90.12


# 2-4) Total_score, Avg_Score, SD_score

In [409]:
s2=sdf_wide.groupby('FinalPIN')['Score'].sum().reset_index()
s2.columns

Index(['FinalPIN', 'Score'], dtype='object')

In [403]:
s3=sdf_wide.groupby('FinalPIN')['Score'].mean().reset_index()
s3

,FinalPIN,Score
0,MAP03001,0.82
1,MAP03002,0.58
2,MAP03003,0.85
3,MAP03004,0.94
4,MAP03005,0.94
...,...,...
837,WPX08020,1.00
838,WPX08021,0.99
839,WPX08022,0.92
840,WPX10003,0.96


In [404]:
s4=sdf_wide.groupby('FinalPIN')['Score'].std().reset_index()
s4

,FinalPIN,Score
0,MAP03001,0.40
1,MAP03002,0.50
2,MAP03003,0.36
3,MAP03004,0.24
4,MAP03005,0.24
...,...,...
837,WPX08020,0.00
838,WPX08021,0.12
839,WPX08022,0.28
840,WPX10003,0.19


# 5.6) Avg_RT_All, SD_RT_All

In [411]:
s5=sdf_wide.groupby(['FinalPIN'])['ResponseTime'].mean().reset_index()
s5

,FinalPIN,ResponseTime
0,MAP03001,9.63
1,MAP03002,2.53
2,MAP03003,3.43
3,MAP03004,5.68
4,MAP03005,5.49
...,...,...
837,WPX08020,2.37
838,WPX08021,1.34
839,WPX08022,1.88
840,WPX10003,1.61


In [412]:
s6=sdf_wide.groupby(['FinalPIN'])['ResponseTime'].std().reset_index()
s6

,FinalPIN,ResponseTime
0,MAP03001,4.45
1,MAP03002,1.11
2,MAP03003,1.15
3,MAP03004,4.18
4,MAP03005,2.09
...,...,...
837,WPX08020,0.88
838,WPX08021,0.36
839,WPX08022,0.75
840,WPX10003,0.52


# 7) Avg_RT_Correct, SD_RT_Correct

In [384]:
correct =sdf_wide[sdf_wide.Score==1]
wrong= sdf_wide[sdf_wide.Score==0]

In [413]:
s7=correct.groupby('FinalPIN')['ResponseTime'].mean().reset_index()
s7

,FinalPIN,ResponseTime
0,MAP03001,9.84
1,MAP03002,2.96
2,MAP03003,3.63
3,MAP03004,5.86
4,MAP03005,5.52
...,...,...
837,WPX08020,2.37
838,WPX08021,1.34
839,WPX08022,1.92
840,WPX10003,1.65


In [414]:
s8=correct.groupby('FinalPIN')['ResponseTime'].std().reset_index()
s8

,FinalPIN,ResponseTime
0,MAP03001,4.90
1,MAP03002,1.20
2,MAP03003,1.12
3,MAP03004,4.24
4,MAP03005,2.15
...,...,...
837,WPX08020,0.88
838,WPX08021,0.36
839,WPX08022,0.75
840,WPX10003,0.49


# 9-10) Avg_RT_Incorrect, SD_RT_Incorrect

In [415]:
s9=wrong.groupby('FinalPIN')['ResponseTime'].mean().reset_index()
s9

,FinalPIN,ResponseTime
0,MAP03001,8.69
1,MAP03002,1.94
2,MAP03003,2.30
3,MAP03004,2.63
4,MAP03005,4.99
...,...,...
594,WPX08019,1.73
595,WPX08021,1.18
596,WPX08022,1.37
597,WPX10003,0.60


In [416]:
s10=wrong.groupby('FinalPIN')['ResponseTime'].std().reset_index()
s10

,FinalPIN,ResponseTime
0,MAP03001,1.86
1,MAP03002,0.62
2,MAP03003,0.54
3,MAP03004,NaN
4,MAP03005,NaN
...,...,...
594,WPX08019,NaN
595,WPX08021,NaN
596,WPX08022,0.44
597,WPX10003,0.54


In [425]:
var = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10]

In [427]:
from functools import reduce
sdf_final = reduce(lambda left,right: pd.merge(left,right,on='FinalPIN'), var)

In [430]:
sdf_final.columns=['FinalPIN','age_months_at_test', 'ResponseTimes','Total_score','Avg_score','SD_score', 'Avg_RT_All','SD_RT_All',
                  'Avg_RT_Correct', 'SD_RT_Correct','Avg_RT_Incorrect', 'SD_RT_Incorrect']

In [431]:
sdf_final.head()

,FinalPIN,age_months_at_test,ResponseTimes,Total_score,Avg_score,SD_score,Avg_RT_All,SD_RT_All,Avg_RT_Correct,SD_RT_Correct,Avg_RT_Incorrect,SD_RT_Incorrect
0,MAP03001,43.07,105.94,9,0.82,0.40,9.63,4.45,9.84,4.90,8.69,1.86
1,MAP03002,41.82,91.20,21,0.58,0.50,2.53,1.11,2.96,1.20,1.94,0.62
2,MAP03003,43.73,92.61,23,0.85,0.36,3.43,1.15,3.63,1.12,2.30,0.54
3,MAP03004,37.49,102.17,17,0.94,0.24,5.68,4.18,5.86,4.24,2.63,NaN
4,MAP03005,39.75,93.32,16,0.94,0.24,5.49,2.09,5.52,2.15,4.99,NaN


In [434]:
def get_Time_Num(data):

    nlist =data.groupby('FinalPIN')['ItemID'].count().reset_index()
    nlist.columns =['FinalPIN', 'N_Live']
    
    first=data.groupby('FinalPIN')['ItemID','DateCreated'].first()
    last =data.groupby('FinalPIN')['ItemID','DateCreated'].last()
    
    t1 = pd.merge(first, last, on='FinalPIN', how='outer')
    t1.columns =['ItemID_first', 'DateCreated_first', 'ItemID_last', 'DateCreated_last']
    #Calculate the duration
    t1['Time_Total'] =pd.to_datetime(t1['DateCreated_last'], format='%Y-%m-%d %H:%M:%S')-pd.to_datetime(t1['DateCreated_first'], format='%Y-%m-%d %H:%M:%S')
    rv = pd.merge(t1, nlist, on='FinalPIN',how='outer')
    display(rv.head())  
    
    return rv

In [440]:
s12 = get_Time_Num(sdf_wide)

,FinalPIN,ItemID_first,DateCreated_first,ItemID_last,DateCreated_last,Time_Total,N_Live
0,MAP03001,SpdM001,2021-06-09 20:17:13,SpdM011,2021-06-09 20:18:48,0 days 00:01:35,11
1,MAP03002,SpdM001,2021-06-08 10:07:41,SpdM036,2021-06-08 10:09:29,0 days 00:01:48,36
2,MAP03003,SpdM001,2021-06-11 10:10:00,SpdM027,2021-06-11 10:11:40,0 days 00:01:40,27
3,MAP03004,SpdM001,2021-06-20 15:51:10,SpdM018,2021-06-20 15:52:47,0 days 00:01:37,18
4,MAP03005,SpdM001,2021-06-08 15:21:23,SpdM017,2021-06-08 15:22:55,0 days 00:01:32,17


In [441]:
srv = pd.merge(sdf_final, s12[['FinalPIN','Time_Total','N_Live']], on='FinalPIN', how='outer')

In [442]:
srv.head()

,FinalPIN,age_months_at_test,ResponseTimes,Total_score,Avg_score,SD_score,Avg_RT_All,SD_RT_All,Avg_RT_Correct,SD_RT_Correct,Avg_RT_Incorrect,SD_RT_Incorrect,Time_Total,N_Live
0,MAP03001,43.07,105.94,9.00,0.82,0.40,9.63,4.45,9.84,4.90,8.69,1.86,0 days 00:01:35,11
1,MAP03002,41.82,91.20,21.00,0.58,0.50,2.53,1.11,2.96,1.20,1.94,0.62,0 days 00:01:48,36
2,MAP03003,43.73,92.61,23.00,0.85,0.36,3.43,1.15,3.63,1.12,2.30,0.54,0 days 00:01:40,27
3,MAP03004,37.49,102.17,17.00,0.94,0.24,5.68,4.18,5.86,4.24,2.63,NaN,0 days 00:01:37,18
4,MAP03005,39.75,93.32,16.00,0.94,0.24,5.49,2.09,5.52,2.15,4.99,NaN,0 days 00:01:32,17


In [443]:
srv.to_csv('C:/Users/tsb7592/Downloads/SpdMtch_part1.csv', index=False)

## Part 2

In [449]:
srv['age'] =srv['FinalPIN'].str.slice(3, 5)

In [450]:
#a. How many unique PINs
srv.groupby('age')['FinalPIN'].nunique()

age
03    109
04    146
05    151
06    142
07    145
08    146
10      1
16      1
H0      1
Name: FinalPIN, dtype: int64

In [452]:
#b. What is the average response time
srv.groupby('age')['ResponseTimes'].mean()

age
03    91.40
04    92.17
05    90.79
06    90.79
07    94.62
08    91.21
10    90.12
16    90.06
H0   100.59
Name: ResponseTimes, dtype: float64